In [1]:
from keras import backend as K 
from keras.layers import Layer

In [2]:
class BaconStackLayer(Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(BaconStackLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        self.kernel = self.add_weight(name = 'kernel',
            shape = (input_shape[1], self.output_dim),
            initializer = 'normal', trainable = True)
        super(BaconStackLayer, self).build(input_shape)
    def call(self, input_data):
        return K.dot(input_data, self.kernel)
    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)
    

In [4]:
from keras.models import Sequential 
from keras.layers import Dense 

model = Sequential() 
model.add(BaconStackLayer(32, input_shape = (16,))) 
model.add(Dense(8, activation = 'softmax')) 
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bacon_stack_layer (BaconSta  (None, 32)               512       
 ckLayer)                                                        
                                                                 
 dense (Dense)               (None, 8)                 264       
                                                                 
Total params: 776
Trainable params: 776
Non-trainable params: 0
_________________________________________________________________


In [24]:
from abc import abstractmethod

from abc import ABC
import tensorflow as tf
from tensorflow import keras
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
import numpy as np
import keras

class baconNet(ABC):
    def __custom_activation(self, x):
        return x
    def __init__(self, size, constantTerm=True):
        get_custom_objects().update(
            {'custom_activation': Activation(self.__custom_activation)})
        self.__model = tf.keras.models.Sequential()
        self.__model.add(tf.keras.Input(shape=(size,)))
        self.__model.add(tf.keras.layers.Dense(
            size, activation=self.__custom_activation))
        self.__model.add(tf.keras.layers.Dense(
            1, activation=self.__custom_activation))
        self.__model.compile(
            optimizer='adam', loss='mean_squared_error', metrics=['mae'])
        self.__size = size
        self.__constantTerm = constantTerm
    @abstractmethod
    def explain(self, singleVariable=False):
        pass

    @abstractmethod
    def get_contribution(self, a, b):
        m = np.matmul(
            self.__model.layers[0].weights[0], self.__model.layers[1].weights[0])
        if self.__constantTerm:
            # assuming the last input is a constant column
            bias0 = self.__model.layers[0].bias.numpy()
            weight0 = self.__model.layers[0].weights[0][self.__size-1]
            for i in range(len(bias0)):
                bias0[i] = bias0[i] + weight0[i]
            weight1 = self.__model.layers[1].weights[0]
            for i in range(len(bias0)):
                bias0[i] = bias0[i] * weight1[i][0]
            c = bias0.sum() + self.__model.layers[1].bias.numpy()[0]
        else:
            c = 0
        return m, c
    @abstractmethod
    def expand(self, a, b, y):
        pass

    def predict(self, a, b=0):
        simple = False
        if isinstance(a, list):
            if isinstance(b, list):
                x, X, y = self.expand(a, b, np.zeros(len(a)))
            else:
                x, X, y = self.expand(a, np.zeros(len(a)), np.zeros(len(a)))
        else:
            simple = True
            x, X, y = self.expand([a], [b], [0])
        if simple:
            return self.__model.predict(X)[0][0]
        else:
            return self.__model.predict(X).flatten()

    def fit(self, a, b, y):
        callback = tf.keras.callbacks.EarlyStopping(monitor='mae', patience=20)
        x, X, y = self.expand(a, b, y)
        history = self.__model.fit(
            X, y, epochs=600, batch_size=10, verbose=0, callbacks=[callback])
        return history
    def get_model(self):
        return self.__model
class poly2(baconNet):
    def __init__(self):
        super().__init__(6)

    def get_contribution(self, a, b):
        return super().get_contribution(a, b)

    def explain(self, singleVariable=False):
        a = 1
        b = 1
        if singleVariable:
            b = 0
        m, c = self.get_contribution(a, b)
        delta = 0.01
        terms = []
        if abs(m[2][0]) > delta and abs(a) > delta:
            terms.append(str(round(m[2][0], 4)) + "x^2")
        if abs(m[4][0]) > delta and abs(b*a) > delta:
            terms.append(str(round(m[4][0], 4)) + "xy")
        if abs(m[3][0]) > delta and abs(b) > delta:
            terms.append(str(round(m[3][0], 4)) + "y^2")
        if abs(m[0][0]) > delta and abs(a) > delta:
            terms.append(str(round(m[0][0], 4)) + "x")
        if abs(m[1][0]) > delta and abs(b) > delta:
            terms.append(str(round(m[1][0], 4)) + "y")
        if abs(m[5][0]) > delta and abs(c) > delta:
            terms.append(str(round(c, 4)))
        return "z = " + " + ".join(terms)

    def expand(self, a, b, y):
        X = np.column_stack((a, b,
                             np.power(a, 2),
                             np.power(b, 2),
                             np.product(np.array([a, b]), axis=0),
                             np.ones(len(a))))
        return np.column_stack((a, b)), X, y

class baconStack():
    def __init__(self, bacons):
        for bacon in bacons:
            pass
        inputs0 = keras.Input(shape=(6,))
        x = poly2().get_model()(inputs0)
        inputs1 = keras.Input(shape=(5,))
        inputs2 = keras.Input(shape=(5,))
        x = layers.concatenate([x, inputs1])
        x = poly2().get_model()(x)
        x = layers.concatenate([x, inputs2])
        x = poly2().get_model()(x)
        self.__model = keras.Model(inputs=[inputs0, inputs1, inputs2], outputs=[x], name="stack")
        plot_model(self.__model, "multi_input_and_output_model.png", show_shapes=True)


In [25]:
stack = baconStack([])

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model

from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from IPython.core.display import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [45]:
from sympy import *

AttributeError: module 'sympy' has no attribute 'external'